In [31]:
import sys
sys.path.append('../..')
from config.configs import ROOT_DIR
import os

file_path = ROOT_DIR / 'data' / 'mark' / 'Fw__Examples_of_QC_reports_for_setting_up_templates_in_Leadtools' / 'PO 177650 Fresh All Green Egypt Strawberries Intake Report.pdf'
file_path

WindowsPath('d:/github/OCR/notebooks/openai/../../data/mark/Fw__Examples_of_QC_reports_for_setting_up_templates_in_Leadtools/PO 177650 Fresh All Green Egypt Strawberries Intake Report.pdf')

In [35]:
from openai import OpenAI

# Create a client
client = OpenAI()

# Upload the PDF file
file = client.files.create(
    file=open(file_path, "rb"),
    purpose="assistants"
)

In [ ]:
assistant = client.beta.assistants.create(
    name="PDF Summarizer",
    description="You are adept at summarizing PDF reports. You analyze data present in .pdf files, understand trends, and provide concise summaries.",
    model="gpt-4o",
    tools=[{"type": "file_search"}]
)

In [39]:
assistant

Assistant(id='asst_pGTouHhRPgAHdbtc8hAltgSw', created_at=1738859194, description='You are adept at summarizing PDF reports. You analyze data present in .pdf files, understand trends, and provide concise summaries.', instructions=None, metadata={}, model='gpt-4o', name='PDF Summarizer', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0, reasoning_effort=None)